<a href="https://colab.research.google.com/github/eynomr/Course-Planner/blob/main/CPSC481_Project_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
from collections import defaultdict, deque, Counter
from itertools import combinations

Problem Class Definition

In [ ]:
class Problem(object):
    """
    The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states
    (or give an `is_goal` method) and perhaps other keyword args for the subclass.
    """

    def __init__(self, initial=None, goal=None, **kwds):
        self.__dict__.update(initial=initial, goal=goal, **kwds)

    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0

    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

Node Class Definition

In [ ]:
class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost


failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.

In [ ]:
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)


def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None):
        return []
    return path_states(node.parent) + [node.state]

In [ ]:
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x):
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)

    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]

    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

Search Functions

In [ ]:
def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure


def best_first_tree_search(problem, f):
    "A version of best_first_search without the `reached` table."
    frontier = PriorityQueue([Node(problem.initial)], key=f)
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            if not is_cycle(child):
                frontier.add(child)
    return failure


def g(n): return n.path_cost


def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + h(n))


def astar_tree_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n), with no `reached` table."""
    h = h or problem.h
    return best_first_tree_search(problem, f=lambda n: g(n) + h(n))


def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)

In [4]:
df = pd.read_csv('./data/course_data.csv')

In [5]:
df

,Unnamed: 0,course_number,course_name,description,work_load,prerequisits
0,0,CPSC 120,Introduction to Programming (3),Introduction to the concepts underlying all co...,"1.5 hours lecture, 3 hours laboratory",['MATH 125.']
1,1,CPSC 121,Object-Oriented Programming (3),The object-oriented programming paradigm: clas...,"2 hours lecture, 2 hours activity",['CPSC 120 or passing score on Computer Scienc...
2,2,CPSC 131,Data Structures (3),"Classical data structures: vector, linked list...","Classical data structures: vector, linked list...",['CPSC 121 or sufficient score on the Computer...
3,3,CPSC 223C,C Programming (3),"Systems programming in the C language, includi...","Systems programming in the C language, includi...",['CPSC 131.']
4,4,CPSC 223J,Java Programming (3),"Characteristics of Java: portable, robust, sec...","2 hours lecture, 2 hours lab per week",['CPSC 131.']
...,...,...,...,...,...,...
75,75,MATH 335,Mathematical Probability (4),"Probability theory; discrete, continuous and m...","Probability theory; discrete, continuous and m...",['MATH 250A with a “C” (2.0) or better.']
76,76,MATH 340,Numerical Analysis (4),Approximate numerical solutions of systems of ...,Approximate numerical solutions of systems of ...,"['MATH 250A', 'MATH 207 or MATH 250B', 'MATH 1..."
77,77,MATH 370,Mathematical Model Building (4),Introduction to mathematical models in science...,Introduction to mathematical models in science...,"['MATH 250A', 'MATH 207 or MATH 250B', 'MATH 1..."
78,78,HONR 201B,Honors Seminar: American Institutions and Valu...,Critically examines the development of America...,Critically examines the development of America...,['University Honors Program student.']
